# Laboratorio 3 — Clustering: segmentación de clientes (no supervisado)

Aplicarás K-Means y métodos de evaluación para elegir número de clusters sin etiquetas.

## Objetivos
- Entender el clustering como exploración y segmentación.
- Aplicar escalado + K-Means con selección de k.
- Interpretar clusters y proponer acciones.

## Requisitos
- scikit-learn
- matplotlib
- pandas


## Contexto

El clustering es útil cuando no hay etiqueta (no hay 'verdad' supervisada).
Sirve para *descubrir* estructura: segmentos, comportamientos, grupos de riesgo, etc.

En esta práctica simularemos clientes con variables de comportamiento.


## 1) Dataset simulado

Generaremos un dataset con: frecuencia de compra, gasto medio, antigüedad y visitas web.
Después, veremos si aparecen grupos coherentes.


In [ ]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(42)
n = 2000

# 4 segmentos simulados
seg = rng.choice([0,1,2,3], size=n, p=[0.25,0.30,0.25,0.20])

def noise(scale, size): 
    return rng.normal(0, scale, size)

freq = np.select(
    [seg==0, seg==1, seg==2, seg==3],
    [rng.normal(2, 0.6, n), rng.normal(8, 1.5, n), rng.normal(5, 1.0, n), rng.normal(12, 2.0, n)]
).clip(0.1)

ticket = np.select(
    [seg==0, seg==1, seg==2, seg==3],
    [rng.normal(18, 6, n), rng.normal(45, 12, n), rng.normal(70, 18, n), rng.normal(28, 8, n)]
).clip(1)

antig = np.select(
    [seg==0, seg==1, seg==2, seg==3],
    [rng.normal(4, 2, n), rng.normal(18, 7, n), rng.normal(10, 4, n), rng.normal(30, 10, n)]
).clip(0)

visitas = np.select(
    [seg==0, seg==1, seg==2, seg==3],
    [rng.normal(6, 2, n), rng.normal(20, 6, n), rng.normal(12, 4, n), rng.normal(28, 8, n)]
).clip(0)

df = pd.DataFrame({
    "compras_mes": freq,
    "ticket_medio": ticket,
    "antiguedad_meses": antig,
    "visitas_web_mes": visitas
})
df.head()


## 2) Escalado + K-Means

K-Means es sensible a escalas (euclídea). Por eso escalamos.


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("kmeans", KMeans(n_clusters=4, n_init="auto", random_state=42))
])

labels = pipe.fit_predict(df)
df_assign = df.copy()
df_assign["cluster"] = labels
df_assign["cluster"].value_counts()


## 3) Elegir k: método del codo y silhouette

En no supervisado, elegimos `k` por criterios internos. Dos habituales:
- Inercia (codo)
- Silhouette (cohesión/separación)

No hay un número 'mágico': se combina con interpretabilidad y uso final.


In [ ]:
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

ks = range(2, 11)
inertias = []
sil = []

X_scaled = StandardScaler().fit_transform(df)

for k in ks:
    km = KMeans(n_clusters=k, n_init="auto", random_state=42)
    lbl = km.fit_predict(X_scaled)
    inertias.append(km.inertia_)
    sil.append(silhouette_score(X_scaled, lbl))

plt.figure(figsize=(7,4))
plt.plot(list(ks), inertias, marker="o")
plt.title("Método del codo (inercia)")
plt.xlabel("k")
plt.ylabel("Inercia")
plt.show()

plt.figure(figsize=(7,4))
plt.plot(list(ks), sil, marker="o")
plt.title("Silhouette vs k")
plt.xlabel("k")
plt.ylabel("Silhouette")
plt.show()


## 4) Interpretar segmentos

Una manera práctica: medias por cluster y descripción cualitativa.
Después, traducirlo a acciones (marketing, retención, riesgo, etc.).


In [ ]:
summary = df_assign.groupby("cluster").mean().round(2)
summary


## 5) Visualización 2D con PCA

Reducimos a 2 componentes solo para visualizar (no para entrenar).


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2, random_state=42)
coords = pca.fit_transform(X_scaled)

viz = pd.DataFrame(coords, columns=["pc1","pc2"])
viz["cluster"] = labels

import matplotlib.pyplot as plt

plt.figure(figsize=(7,5))
for c in sorted(viz["cluster"].unique()):
    m = viz["cluster"]==c
    plt.scatter(viz.loc[m,"pc1"], viz.loc[m,"pc2"], s=10, label=f"cluster {c}", alpha=0.6)
plt.title("Clusters (PCA 2D)")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
plt.show()


## Ejercicios

### Ejercicio 1: Segmentación accionable
Elige un k razonado (2–8) y escribe un párrafo por cluster describiendo el segmento y una acción concreta (p.ej., campaña, retención, upselling, control de riesgo).

**Entregables**
- k elegido y justificación
- Descripción + acción por cluster

**Criterios de evaluación**
- Justificación coherente con métricas
- Segmentos interpretables
- Acciones específicas y realistas

### Ejercicio 2: Comparar con otro algoritmo
Prueba `DBSCAN` o `GaussianMixture` y compara: ¿qué cambia en número de grupos y en interpretabilidad?

**Entregables**
- Resultado del modelo alternativo
- Comparación cualitativa y, si aplica, silhouette

**Criterios de evaluación**
- Configuración razonada
- Comparación clara
- Se mencionan limitaciones
